In [3]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [4]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [1]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [2]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [3]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [8]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    ################################################ s', Q = env(s, a)
    ################################################ ~s', ~Q = env(s, ~a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels))
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels))
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    _, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                  action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    ###################################################### Q(s,a)= r + Q'(s',a') # max
    ###################################################### ~Q(s,~a)= r # min
    ###################################################### ~Q(s,~a)= r + Q'(s',a') # max
    targetQs = rewards + (gamma * aQs2_logits)
    eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # max
    eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
    aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
    return actions_logits, aloss, eloss, aloss2

In [4]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [5]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [6]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [7]:
env.observation_space, env.action_space

NameError: name 'env' is not defined

In [8]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [16]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [17]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:16.0000 R:16.0000 rate:0.0320 aloss:0.6940 eloss:4.3616 aloss2:3.0810 exploreP:0.9984
Episode:1 meanR:19.0000 R:22.0000 rate:0.0440 aloss:0.6965 eloss:4.3686 aloss2:3.0711 exploreP:0.9962
Episode:2 meanR:21.3333 R:26.0000 rate:0.0520 aloss:0.6949 eloss:4.3494 aloss2:3.0587 exploreP:0.9937
Episode:3 meanR:20.0000 R:16.0000 rate:0.0320 aloss:0.6961 eloss:4.3551 aloss2:3.0367 exploreP:0.9921
Episode:4 meanR:19.8000 R:19.0000 rate:0.0380 aloss:0.6933 eloss:4.3458 aloss2:3.0398 exploreP:0.9902
Episode:5 meanR:20.3333 R:23.0000 rate:0.0460 aloss:0.6922 eloss:4.3354 aloss2:3.0202 exploreP:0.9880
Episode:6 meanR:21.1429 R:26.0000 rate:0.0520 aloss:0.6920 eloss:4.3403 aloss2:3.0161 exploreP:0.9855
Episode:7 meanR:20.6250 R:17.0000 rate:0.0340 aloss:0.6902 eloss:4.3457 aloss2:3.0058 exploreP:0.9838
Episode:8 meanR:22.1111 R:34.0000 rate:0.0680 aloss:0.6903 eloss:4.3325 aloss2:2.9986 exploreP:0.9805
Episode:9 meanR:21.2000 R:13.0000 rate:0.0260 aloss:0.6925 eloss:4.3271 aloss2:2.9

Episode:80 meanR:27.3457 R:24.0000 rate:0.0480 aloss:0.6740 eloss:5.4519 aloss2:2.6319 exploreP:0.8033
Episode:81 meanR:27.5488 R:44.0000 rate:0.0880 aloss:0.6851 eloss:5.4343 aloss2:2.6441 exploreP:0.7998
Episode:82 meanR:27.6145 R:33.0000 rate:0.0660 aloss:0.6720 eloss:5.4099 aloss2:2.6462 exploreP:0.7972
Episode:83 meanR:27.6429 R:30.0000 rate:0.0600 aloss:0.6918 eloss:5.4059 aloss2:2.6406 exploreP:0.7949
Episode:84 meanR:27.4824 R:14.0000 rate:0.0280 aloss:0.6729 eloss:5.3549 aloss2:2.6634 exploreP:0.7938
Episode:85 meanR:27.3953 R:20.0000 rate:0.0400 aloss:0.6838 eloss:5.3557 aloss2:2.6634 exploreP:0.7922
Episode:86 meanR:27.9195 R:73.0000 rate:0.1460 aloss:0.6864 eloss:5.3323 aloss2:2.6724 exploreP:0.7865
Episode:87 meanR:27.7841 R:16.0000 rate:0.0320 aloss:0.6961 eloss:5.2573 aloss2:2.6752 exploreP:0.7853
Episode:88 meanR:27.9551 R:43.0000 rate:0.0860 aloss:0.6793 eloss:5.2828 aloss2:2.6910 exploreP:0.7819
Episode:89 meanR:27.7889 R:13.0000 rate:0.0260 aloss:0.6739 eloss:5.3037 

Episode:159 meanR:30.7500 R:39.0000 rate:0.0780 aloss:0.6909 eloss:4.4984 aloss2:3.1939 exploreP:0.6362
Episode:160 meanR:30.5400 R:9.0000 rate:0.0180 aloss:0.7006 eloss:4.5086 aloss2:3.1828 exploreP:0.6356
Episode:161 meanR:30.6900 R:27.0000 rate:0.0540 aloss:0.7084 eloss:4.5009 aloss2:3.1906 exploreP:0.6339
Episode:162 meanR:30.4400 R:11.0000 rate:0.0220 aloss:0.8404 eloss:4.4820 aloss2:3.1586 exploreP:0.6332
Episode:163 meanR:30.2600 R:12.0000 rate:0.0240 aloss:0.7293 eloss:4.4915 aloss2:3.1904 exploreP:0.6325
Episode:164 meanR:30.1300 R:17.0000 rate:0.0340 aloss:0.7308 eloss:4.4978 aloss2:3.1833 exploreP:0.6314
Episode:165 meanR:30.2600 R:29.0000 rate:0.0580 aloss:0.7442 eloss:4.4887 aloss2:3.1798 exploreP:0.6296
Episode:166 meanR:30.3000 R:44.0000 rate:0.0880 aloss:0.7359 eloss:4.4908 aloss2:3.1832 exploreP:0.6269
Episode:167 meanR:30.7300 R:60.0000 rate:0.1200 aloss:0.7358 eloss:4.4937 aloss2:3.1798 exploreP:0.6232
Episode:168 meanR:30.8500 R:39.0000 rate:0.0780 aloss:0.7024 elos

Episode:238 meanR:27.9000 R:55.0000 rate:0.1100 aloss:0.6918 eloss:4.5935 aloss2:3.0176 exploreP:0.5114
Episode:239 meanR:27.9700 R:31.0000 rate:0.0620 aloss:0.7059 eloss:4.6058 aloss2:3.0057 exploreP:0.5098
Episode:240 meanR:28.0200 R:27.0000 rate:0.0540 aloss:0.6958 eloss:4.6091 aloss2:3.0047 exploreP:0.5085
Episode:241 meanR:27.9200 R:15.0000 rate:0.0300 aloss:0.6706 eloss:4.6083 aloss2:3.0064 exploreP:0.5077
Episode:242 meanR:28.1100 R:47.0000 rate:0.0940 aloss:0.6913 eloss:4.5994 aloss2:2.9929 exploreP:0.5054
Episode:243 meanR:28.4000 R:38.0000 rate:0.0760 aloss:0.6880 eloss:4.6021 aloss2:2.9960 exploreP:0.5035
Episode:244 meanR:28.5200 R:47.0000 rate:0.0940 aloss:0.6770 eloss:4.5979 aloss2:2.9965 exploreP:0.5012
Episode:245 meanR:28.7800 R:59.0000 rate:0.1180 aloss:0.6818 eloss:4.5999 aloss2:2.9890 exploreP:0.4983
Episode:246 meanR:28.7400 R:17.0000 rate:0.0340 aloss:0.6898 eloss:4.6007 aloss2:2.9866 exploreP:0.4975
Episode:247 meanR:28.7900 R:27.0000 rate:0.0540 aloss:0.6790 elo

Episode:317 meanR:44.0900 R:58.0000 rate:0.1160 aloss:0.7616 eloss:4.7079 aloss2:2.8924 exploreP:0.3479
Episode:318 meanR:44.4100 R:48.0000 rate:0.0960 aloss:0.7274 eloss:4.6945 aloss2:2.8969 exploreP:0.3463
Episode:319 meanR:44.9800 R:78.0000 rate:0.1560 aloss:0.7576 eloss:4.7081 aloss2:2.8899 exploreP:0.3437
Episode:320 meanR:45.1700 R:33.0000 rate:0.0660 aloss:0.8214 eloss:4.7338 aloss2:2.8809 exploreP:0.3426
Episode:321 meanR:45.5100 R:43.0000 rate:0.0860 aloss:0.7427 eloss:4.7045 aloss2:2.8888 exploreP:0.3412
Episode:322 meanR:46.1400 R:78.0000 rate:0.1560 aloss:0.7769 eloss:4.7163 aloss2:2.8885 exploreP:0.3386
Episode:323 meanR:46.9100 R:98.0000 rate:0.1960 aloss:0.7706 eloss:4.7149 aloss2:2.8905 exploreP:0.3354
Episode:324 meanR:47.0800 R:35.0000 rate:0.0700 aloss:0.7766 eloss:4.7025 aloss2:2.8920 exploreP:0.3342
Episode:325 meanR:47.2900 R:41.0000 rate:0.0820 aloss:0.7860 eloss:4.7125 aloss2:2.8931 exploreP:0.3329
Episode:326 meanR:47.6400 R:55.0000 rate:0.1100 aloss:0.7508 elo

...

Episode:6500 meanR:56.6000 R:84.0000 rate:0.1680 aloss:0.2936 eloss:4.2088 aloss2:3.4899 exploreP:0.0100
Episode:6501 meanR:56.4400 R:35.0000 rate:0.0700 aloss:0.2948 eloss:4.2138 aloss2:3.5086 exploreP:0.0100
Episode:6502 meanR:56.7000 R:63.0000 rate:0.1260 aloss:0.2872 eloss:4.1897 aloss2:3.4798 exploreP:0.0100
Episode:6503 meanR:56.8400 R:49.0000 rate:0.0980 aloss:0.2894 eloss:4.2052 aloss2:3.5130 exploreP:0.0100
Episode:6504 meanR:57.0600 R:73.0000 rate:0.1460 aloss:0.2986 eloss:4.2070 aloss2:3.5370 exploreP:0.0100
Episode:6505 meanR:56.8800 R:37.0000 rate:0.0740 aloss:0.3053 eloss:4.2043 aloss2:3.5890 exploreP:0.0100
Episode:6506 meanR:56.7500 R:34.0000 rate:0.0680 aloss:0.3379 eloss:4.2132 aloss2:3.7976 exploreP:0.0100
Episode:6507 meanR:57.0500 R:85.0000 rate:0.1700 aloss:0.2870 eloss:4.2068 aloss2:3.4913 exploreP:0.0100
Episode:6508 meanR:56.9800 R:96.0000 rate:0.1920 aloss:0.2845 eloss:4.2016 aloss2:3.4982 exploreP:0.0100
Episode:6509 meanR:56.7600 R:34.0000 rate:0.0680 aloss:

Episode:6578 meanR:55.9400 R:32.0000 rate:0.0640 aloss:0.2741 eloss:4.1888 aloss2:3.4796 exploreP:0.0100
Episode:6579 meanR:55.8900 R:36.0000 rate:0.0720 aloss:0.2931 eloss:4.1767 aloss2:3.6160 exploreP:0.0100
Episode:6580 meanR:55.8100 R:37.0000 rate:0.0740 aloss:0.2995 eloss:4.1852 aloss2:3.5844 exploreP:0.0100
Episode:6581 meanR:55.9200 R:52.0000 rate:0.1040 aloss:0.2879 eloss:4.1993 aloss2:3.5310 exploreP:0.0100
Episode:6582 meanR:55.9600 R:41.0000 rate:0.0820 aloss:0.2865 eloss:4.1864 aloss2:3.5675 exploreP:0.0100
Episode:6583 meanR:56.0000 R:55.0000 rate:0.1100 aloss:0.2838 eloss:4.1956 aloss2:3.5209 exploreP:0.0100
Episode:6584 meanR:56.0200 R:39.0000 rate:0.0780 aloss:0.2836 eloss:4.1878 aloss2:3.5117 exploreP:0.0100
Episode:6585 meanR:55.8900 R:48.0000 rate:0.0960 aloss:0.2754 eloss:4.1810 aloss2:3.6012 exploreP:0.0100
Episode:6586 meanR:55.8400 R:31.0000 rate:0.0620 aloss:0.2818 eloss:4.1772 aloss2:3.5542 exploreP:0.0100
Episode:6587 meanR:55.7700 R:42.0000 rate:0.0840 aloss:

Episode:6656 meanR:53.2800 R:33.0000 rate:0.0660 aloss:0.2547 eloss:4.1840 aloss2:3.7774 exploreP:0.0100
Episode:6657 meanR:53.3000 R:35.0000 rate:0.0700 aloss:0.2792 eloss:4.1710 aloss2:3.8037 exploreP:0.0100
Episode:6658 meanR:52.8000 R:43.0000 rate:0.0860 aloss:0.2609 eloss:4.2023 aloss2:3.7052 exploreP:0.0100
Episode:6659 meanR:52.6000 R:31.0000 rate:0.0620 aloss:0.2779 eloss:4.1615 aloss2:3.7846 exploreP:0.0100
Episode:6660 meanR:52.5300 R:29.0000 rate:0.0580 aloss:0.2753 eloss:4.1635 aloss2:3.8394 exploreP:0.0100
Episode:6661 meanR:52.2400 R:61.0000 rate:0.1220 aloss:0.2577 eloss:4.1668 aloss2:3.7386 exploreP:0.0100
Episode:6662 meanR:52.2100 R:32.0000 rate:0.0640 aloss:0.2770 eloss:4.1640 aloss2:3.7478 exploreP:0.0100
Episode:6663 meanR:51.1900 R:33.0000 rate:0.0660 aloss:0.2537 eloss:4.1575 aloss2:3.7512 exploreP:0.0100
Episode:6664 meanR:50.8500 R:47.0000 rate:0.0940 aloss:0.2870 eloss:4.2104 aloss2:3.8123 exploreP:0.0100
Episode:6665 meanR:50.6900 R:34.0000 rate:0.0680 aloss:

Episode:6735 meanR:49.1300 R:32.0000 rate:0.0640 aloss:0.2296 eloss:4.1732 aloss2:3.7680 exploreP:0.0100
Episode:6736 meanR:48.7800 R:39.0000 rate:0.0780 aloss:0.2649 eloss:4.1736 aloss2:3.9074 exploreP:0.0100
Episode:6737 meanR:48.9300 R:56.0000 rate:0.1120 aloss:0.2653 eloss:4.1662 aloss2:3.8444 exploreP:0.0100
Episode:6738 meanR:48.6700 R:31.0000 rate:0.0620 aloss:0.2546 eloss:4.1476 aloss2:3.7823 exploreP:0.0100
Episode:6739 meanR:48.6900 R:41.0000 rate:0.0820 aloss:0.2321 eloss:4.1484 aloss2:3.7753 exploreP:0.0100
Episode:6740 meanR:48.7800 R:52.0000 rate:0.1040 aloss:0.2296 eloss:4.1569 aloss2:3.7621 exploreP:0.0100
Episode:6741 meanR:48.9100 R:45.0000 rate:0.0900 aloss:0.2655 eloss:4.1554 aloss2:3.8855 exploreP:0.0100
Episode:6742 meanR:48.9500 R:57.0000 rate:0.1140 aloss:0.2419 eloss:4.1509 aloss2:3.8177 exploreP:0.0100
Episode:6743 meanR:48.5900 R:45.0000 rate:0.0900 aloss:0.2459 eloss:4.1486 aloss2:3.7585 exploreP:0.0100
Episode:6744 meanR:48.1700 R:84.0000 rate:0.1680 aloss:

Episode:6813 meanR:52.6600 R:40.0000 rate:0.0800 aloss:0.2286 eloss:4.1488 aloss2:3.8039 exploreP:0.0100
Episode:6814 meanR:53.0500 R:76.0000 rate:0.1520 aloss:0.2417 eloss:4.1497 aloss2:3.8330 exploreP:0.0100
Episode:6815 meanR:53.1200 R:38.0000 rate:0.0760 aloss:0.2230 eloss:4.1771 aloss2:3.8875 exploreP:0.0100
Episode:6816 meanR:53.1600 R:34.0000 rate:0.0680 aloss:0.2499 eloss:4.1797 aloss2:3.9285 exploreP:0.0100
Episode:6817 meanR:53.2600 R:59.0000 rate:0.1180 aloss:0.2344 eloss:4.1487 aloss2:3.8735 exploreP:0.0100
Episode:6818 meanR:53.4400 R:63.0000 rate:0.1260 aloss:0.2435 eloss:4.1511 aloss2:3.8234 exploreP:0.0100
Episode:6819 meanR:53.6300 R:65.0000 rate:0.1300 aloss:0.2998 eloss:4.1844 aloss2:4.0041 exploreP:0.0100
Episode:6820 meanR:53.2200 R:32.0000 rate:0.0640 aloss:0.3550 eloss:4.2190 aloss2:4.3203 exploreP:0.0100
Episode:6821 meanR:53.3000 R:41.0000 rate:0.0820 aloss:0.2382 eloss:4.2197 aloss2:3.8867 exploreP:0.0100


# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
